In [39]:
import csv
import pandas as pd
import hvplot.pandas
import matplotlib as plt
import os
from pprint import pprint
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nelgiriyewithana/most-streamed-spotify-songs-2024")
if not os.path.exists(os.path.join("data", "spotify.csv")):
    file_name = os.listdir(path)[0]
    os.rename(os.path.join(path, file_name), os.path.join("data", "spotify.csv"))
spotify_df = pd.read_csv("data/spotify.csv", encoding='ISO-8859-1')

In [40]:
clean_df = spotify_df.drop(columns=['TIDAL Popularity'])
clean_df = clean_df.dropna()
for i in clean_df.columns:
    clean_df.rename(columns={i: (i.lower()).replace(" ", "_")}, inplace=True)
clean_df.head()

,track,album_name,artist,release_date,isrc,all_time_rank,track_score,spotify_streams,spotify_playlist_count,spotify_playlist_reach,...,airplay_spins,siriusxm_spins,deezer_playlist_count,deezer_playlist_reach,amazon_playlist_count,pandora_streams,pandora_track_stations,soundcloud_streams,shazam_counts,explicit_track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,4/26/2024,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,"40,975",684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",0
1,Not Like Us,Not Like Us,Kendrick Lamar,5/4/2024,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,"40,778",3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",1
2,i like the way you kiss me,I like the way you kiss me,Artemas,3/19/2024,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,"74,333",536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",0
5,Lovin On Me,Lovin On Me,Jack Harlow,11/10/2023,USAT22311371,6,410.1,"670,665,438","105,892","175,421,034",...,"522,042","4,654",86.0,"17,167,254",152.0,"138,529,362","50,982","9,438,601","4,517,131",1
9,BAND4BAND (feat. Lil Baby),BAND4BAND (feat. Lil Baby),Central Cee,5/23/2024,USSM12404354,10,330.6,"90,676,573","10,400","184,199,419",...,"3,823",117,78.0,"10,800,098",92.0,"1,005,626",842,"3,679,709","666,302",1


In [35]:
for col in clean_df.columns:
    print(f"{col}")

Track
Album Name
artist
Release Date
ISRC
All Time Rank
Track Score
Spotify Streams
Spotify Playlist Count
Spotify Playlist Reach
Spotify Popularity
YouTube Views
YouTube Likes
TikTok Posts
TikTok Likes
TikTok Views
YouTube Playlist Reach
Apple Music Playlist Count
AirPlay Spins
SiriusXM Spins
Deezer Playlist Count
Deezer Playlist Reach
Amazon Playlist Count
Pandora Streams
Pandora Track Stations
Soundcloud Streams
Shazam Counts
Explicit Track
